In [74]:
import os
os.chdir("/Users/arpitverma/kidney-disease-Classification-DeepLearning")

In [75]:
%pwd

'/Users/arpitverma/kidney-disease-Classification-DeepLearning'

In [76]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_early_stopping_patience: int
    params_reduce_lr_patience: int
    params_reduce_lr_factor: float
    params_min_lr: float
    params_learning_rate: float

In [77]:
from src.kidneyClassifier.constants import *
from src.kidneyClassifier.utils.common import read_yaml, create_directories

import tensorflow as tf

In [78]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training.training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_early_stopping_patience=params.EARLY_STOPPING_PATIENCE,
            params_reduce_lr_patience=params.REDUCE_LR_PATIENCE,
            params_reduce_lr_factor=params.REDUCE_LR_FACTOR,
            params_min_lr=params.MIN_LR,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config
    


 

In [79]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from collections import Counter
tf.config.run_functions_eagerly(True)

In [80]:


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    
    def get_base_model(self):
        # Load the model WITHOUT compiling (this avoids the optimizer error)
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False  # KEY FIX: Don't load the old optimizer
        )
        
        # Recompile with fresh optimizer
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        print(f"✓ Model loaded and recompiled with learning rate: {self.config.params_learning_rate}")
    
    
    def train_valid_generator(self):
        """
        Creates training and validation generators from a single directory.
        Uses validation_split=0.20 to automatically split data into 80% train and 20% validation.
        """
        
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode='categorical'
        )

        # Validation generator (no augmentation)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Training generator (with augmentation if enabled)
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )


    def compute_class_weights(self):
        """Compute class weights for imbalanced datasets"""
        # Get class labels
        labels = self.train_generator.classes
        
        # Count samples per class
        class_counts = Counter(labels)
        total_samples = len(labels)
        num_classes = len(class_counts)
        
        # Compute balanced class weights: total / (num_classes * count_per_class)
        class_weight_dict = {}
        for class_idx, count in class_counts.items():
            class_weight_dict[class_idx] = total_samples / (num_classes * count)
        
        print("\nClass weights computed:")
        for class_name, class_idx in self.train_generator.class_indices.items():
            print(f"  {class_name} (samples: {class_counts[class_idx]}): weight = {class_weight_dict[class_idx]:.4f}")
        
        return class_weight_dict


    def get_callbacks(self):
        """Create callbacks for training"""
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=self.config.params_early_stopping_patience,
            restore_best_weights=True,
            verbose=1
        )

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=self.config.params_reduce_lr_factor,
            patience=self.config.params_reduce_lr_patience,
            min_lr=self.config.params_min_lr,
            verbose=1
        )

        return [early_stopping, reduce_lr]


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Compute class weights
        class_weight_dict = self.compute_class_weights()
        
        # Get callbacks
        callbacks = self.get_callbacks()

        # Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            class_weight=class_weight_dict,
            callbacks=callbacks
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [ ]:


try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    print(f"\n✗ Error: {e}")
    raise e

[2025-11-08 20:31:45,014: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-11-08 20:31:45,016: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-08 20:31:45,016: INFO: common: created directory at: artifacts]
[2025-11-08 20:31:45,017: INFO: common: created directory at: artifacts/training]
✓ Model loaded and recompiled with learning rate: 1e-05
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.

Dataset Summary:
Classes: ['Cyst', 'Normal', 'Stone', 'Tumor']
Training samples: 9959
Validation samples: 2487
Steps per epoch: 622


Class weights computed:
  Cyst (samples: 2968): weight = 0.8389
  Normal (samples: 4062): weight = 0.6129
  Stone (samples: 1102): weight = 2.2593
  Tumor (samples: 1827): weight = 1.3628
622/622 ━━━━━━━━━━━━━━━━━━━━ 1373s 2s/step - accuracy: 0.5545 - loss: 6.4510 - val_accuracy: 0.1911 - val_loss: 8.4584 - learning_rate: 1.0000e-05
Restoring model weights from the end of the best epoch: 1.
[